In [1]:
# Cell 1: Import Libraries and Database Configuration
import pandas as pd
import numpy as np
from pathlib import Path
import warnings
import math
from sqlalchemy import create_engine
import pymysql
import os
from tqdm.notebook import tqdm

warnings.filterwarnings('ignore')
tqdm.pandas()
print("Libraries imported and connection configured.")

# Database Configuration 
db_config = {
    'host': '127.0.0.1',
    'user': 'root',
    'password': '',
    'database': 'trading_system'
}
db_url = f"mysql+pymysql://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}"
engine = create_engine(db_url)

# Define Base Directory
base_dir = Path.cwd().parent
print(f"Project Base Directory: {base_dir}")

Libraries imported and connection configured.
Project Base Directory: c:\Users\18kyu\Desktop\Unishit\IR


In [2]:
# Cell 2: Load Data from Database
print("Loading data from database.")

# 1. Load Stock Prices
print("Loading 'stock_prices'.")
try:
    sql_prices = """
    SELECT 
        ticker, date, 
        CAST(close AS DECIMAL(10, 4)) AS close, 
        CAST(volume AS SIGNED) AS volume 
    FROM stock_prices
    """
    df_prices = pd.read_sql(sql_prices, con=engine)
    df_prices['date'] = pd.to_datetime(df_prices['date'], errors='coerce').dt.date
    df_prices = df_prices.dropna(subset=['date', 'close', 'volume'])
    print(f"Loaded {len(df_prices):,} price records.")

except Exception as e:
    print(f"Error loading prices: {e}")

# 2. Load Headlines
print("Loading 'headlines'.")
try:
    sql_headlines = "SELECT id, date, stock, headline FROM headlines"
    df_headlines = pd.read_sql(sql_headlines, con=engine, index_col='id')
    df_headlines['date'] = pd.to_datetime(df_headlines['date'], errors='coerce').dt.date
    df_headlines = df_headlines.dropna(subset=['date'])
    print(f"Loaded {len(df_headlines):,} headline records.")
except Exception as e:
    print(f"Error loading headlines: {e}")

# 3. Load Sentiment Scores
print("Loading 'sentiment_scores'.")
try:
    sql_scores = "SELECT headline_id, textblob_polarity, vader_compound, finbert_compound FROM sentiment_scores"
    df_scores = pd.read_sql(sql_scores, con=engine, index_col='headline_id')
    print(f"Loaded {len(df_scores):,} score records.")
except Exception as e:
    print(f"Error loading scores: {e}")

print("\n All data loaded successfully.")

Loading data from database.
Loading 'stock_prices'.
Loaded 8,986,458 price records.
Loading 'headlines'.
Loaded 1,147,268 headline records.
Loading 'sentiment_scores'.
Loaded 1,141,860 score records.

 All data loaded successfully.


In [3]:
# Cell 3: Impute Sentiment Scores and Merge with Prices
print("Merging scores with headlines.")

# 1. Join scores with headlines
df_sentiment_daily = df_headlines.join(df_scores, how='inner')
print(f"Merged {len(df_sentiment_daily)} sentiment records with dates/stocks.")

# 2. Aggregate sentiment by day
print("Aggregating sentiment scores by stock and day.")
df_sentiment_agg = df_sentiment_daily.groupby(['stock', 'date']).agg(
    textblob_polarity = ('textblob_polarity', 'mean'),
    vader_compound = ('vader_compound', 'mean'),
    finbert_compound = ('finbert_compound', 'mean'),
    news_count = ('headline', 'count')
).reset_index()
print(f"Created {len(df_sentiment_agg)} daily aggregated sentiment records.")

# 3. Merge daily sentiment with prices
print("\nMerging daily prices and sentiment.")
merged_daily_df = pd.merge(
    df_prices,
    df_sentiment_agg,
    left_on=['ticker', 'date'],
    right_on=['stock', 'date'],
    how='left'
)

# 4. Impute missing sentiment scores
merged_daily_df = merged_daily_df.drop(columns=['stock'])
merged_daily_df = merged_daily_df.sort_values(by=['ticker', 'date']).reset_index(drop=True)

print("Performing imputation of missing sentiment scores.")

sentiment_cols = ['textblob_polarity', 'vader_compound', 'finbert_compound']

def smart_sentiment_imputation(group, decay_window=21, baseline_window=90):
    group = group.copy()
    group = group.reset_index(drop=True) 
    
    # Create 'has_news' flag
    group['has_news'] = (group['news_count'] > 0).fillna(False).astype(int)
    group['days_since_news'] = decay_window
    
    for col in sentiment_cols:
        # Step 1: Calculate stock's historical baseline
        news_only = group.loc[group['has_news'] == 1, col].copy()
        news_only_baseline = news_only.rolling(
            window=baseline_window, min_periods=5
        ).mean()
        
        group[f'{col}_baseline'] = np.nan
        group.loc[group['has_news'] == 1, f'{col}_baseline'] = news_only_baseline.values
        group[f'{col}_baseline'] = group[f'{col}_baseline'].ffill()
        
        # Step 2: Apply decay
        imputed_values = group[col].copy()
        last_news_idx = None
        last_news_value = None
        
        for i in range(len(group)):
            if group['has_news'].iloc[i] == 1 and pd.notna(group[col].iloc[i]):
                last_news_idx = i
                last_news_value = group[col].iloc[i]
                group.loc[i, 'days_since_news'] = 0 # Reset counter
            elif last_news_idx is not None:
                # We have seen news before
                days_since = i - last_news_idx
                group.loc[i, 'days_since_news'] = min(days_since, decay_window)
                
                if pd.isna(group[col].iloc[i]):
                    if days_since <= decay_window:
                        decay_factor = np.exp(-days_since / 7)
                        baseline = group[f'{col}_baseline'].iloc[i]
                        
                        if pd.notna(baseline):
                            imputed_values.iloc[i] = (decay_factor * last_news_value + 
                                                     (1 - decay_factor) * baseline)
                        else:
                            imputed_values.iloc[i] = decay_factor * last_news_value
                    else:
                        baseline = group[f'{col}_baseline'].iloc[i]
                        if pd.notna(baseline):
                            imputed_values.iloc[i] = baseline
            else:
                pass
                    
        group[col] = imputed_values
    
    group = group.drop(columns=[f'{col}_baseline' for col in sentiment_cols])
    return group

# Apply smart imputation
print("Processing sentiment imputation by ticker.")
merged_daily_df = merged_daily_df.groupby('ticker', group_keys=False).progress_apply(
    lambda x: smart_sentiment_imputation(x, decay_window=21, baseline_window=90)
)

# 5. Calculate market-wide baseline for any remaining NaNs
print("\nCalculating market-wide sentiment baseline.")
for col in sentiment_cols:
    market_baseline = merged_daily_df.loc[
        merged_daily_df['has_news'] == 1, col
    ].mean()
    
    remaining_nas = merged_daily_df[col].isna().sum()
    if remaining_nas > 0:
        print(f"  Filling {remaining_nas} remaining NaNs in {col} with market baseline: {market_baseline:.4f}")
        merged_daily_df[col] = merged_daily_df[col].fillna(market_baseline)

# Fill news_count
merged_daily_df['news_count'] = merged_daily_df['news_count'].fillna(0)

# 6. Add sentiment features
print("\nCreating sentiment quality features.")
merged_daily_df['sentiment_freshness'] = np.exp(-merged_daily_df['days_since_news'] / 7)

for col in sentiment_cols:
    merged_daily_df[f'{col}_volatility'] = merged_daily_df.groupby('ticker')[col].transform(
        lambda x: x.rolling(window=30, min_periods=5).std()
    )
    merged_daily_df[f'{col}_volatility'] = merged_daily_df[f'{col}_volatility'].fillna(
        merged_daily_df[f'{col}_volatility'].median()
    )

# 7. Convert date back to datetime
merged_daily_df['date'] = pd.to_datetime(merged_daily_df['date'])
print(f"\nCreated final daily merged dataset with {len(merged_daily_df)} rows.")

# 8. Analyze imputation results
print("\n=== Imputation Quality Analysis ===")
for col in sentiment_cols:
    non_zero = (merged_daily_df[col] != 0).sum()
    pct_non_zero = (non_zero / len(merged_daily_df)) * 100
    mean_val = merged_daily_df[col].mean()
    std_val = merged_daily_df[col].std()
    print(f"{col}:")
    print(f"  Non-zero values: {non_zero:,} ({pct_non_zero:.1f}%)")
    print(f"  Mean: {mean_val:.4f}")
    print(f"  Std: {std_val:.4f}")

print(f"\nDays with actual news: {merged_daily_df['has_news'].sum():,} ({(merged_daily_df['has_news'].sum()/len(merged_daily_df)*100):.1f}%)")

print("\nSample data (AAL ticker):")
sample_df = merged_daily_df.query("ticker == 'AAL' and date >= '2010-01-04'").head(20)
print(sample_df[['ticker', 'date', 'close', 'finbert_compound', 'has_news', 
                 'days_since_news', 'sentiment_freshness']].to_string())

Merging scores with headlines.
Merged 1141860 sentiment records with dates/stocks.
Aggregating sentiment scores by stock and day.
Created 786414 daily aggregated sentiment records.

Merging daily prices and sentiment.
Performing imputation of missing sentiment scores.
Processing sentiment imputation by ticker.


  0%|          | 0/4041 [00:00<?, ?it/s]


Calculating market-wide sentiment baseline.
  Filling 4133432 remaining NaNs in textblob_polarity with market baseline: 0.0530
  Filling 4133432 remaining NaNs in vader_compound with market baseline: 0.0891
  Filling 4133432 remaining NaNs in finbert_compound with market baseline: 0.0383

Creating sentiment quality features.

Created final daily merged dataset with 8986458 rows.

=== Imputation Quality Analysis ===
textblob_polarity:
  Non-zero values: 8,429,893 (93.8%)
  Mean: 0.0474
  Std: 0.0944
vader_compound:
  Non-zero values: 8,593,816 (95.6%)
  Mean: 0.0905
  Std: 0.1265
finbert_compound:
  Non-zero values: 8,348,197 (92.9%)
  Mean: 0.0275
  Std: 0.1915

Days with actual news: 685,462 (7.6%)

Sample data (AAL ticker):
   ticker       date   close  finbert_compound  has_news  days_since_news  sentiment_freshness
0     AAL 2010-01-04  4.4969          0.038326         0               21             0.049787
1     AAL 2010-01-05  5.0060          0.038326         0               21

In [4]:
# Cell 4: Create Weekly Tabular Dataset
print("Creating Weekly Tabular Dataset.")

# 1. Define Feature Functions
def calculate_rsi(prices, window=14):
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def create_weekly_features(group):
    # a) Resample daily data to weekly
    group = group.set_index('date')
    
    # Add all sentiment and technical features to the aggregation dictionary
    weekly_group = group.resample('W').agg(
        # Price
        adj_close = ('close', 'last'), 
        volume = ('volume', 'sum'),
        
        # Base Sentiment
        textblob_polarity = ('textblob_polarity', 'mean'),
        vader_compound = ('vader_compound', 'mean'),
        finbert_compound = ('finbert_compound', 'mean'),
        
        # News Count / Freshness
        news_count = ('news_count', 'sum'),
        has_news = ('has_news', 'max'), # 1 if any news in week
        days_since_news = ('days_since_news', 'mean'), # Avg staleness
        sentiment_freshness = ('sentiment_freshness', 'mean'),
        
        # Sentiment Volatility
        textblob_polarity_volatility = ('textblob_polarity_volatility', 'mean'),
        vader_compound_volatility = ('vader_compound_volatility', 'mean'),
        finbert_compound_volatility = ('finbert_compound_volatility', 'mean')
    )
    
    # Drop weeks with no trading
    weekly_group = weekly_group.dropna(subset=['adj_close']) 
    
    # b) Create technical features 
    df = weekly_group.copy()
    df['prev_close'] = df['adj_close'].shift(1)
    df['return'] = df['adj_close'].pct_change()
    df['log_return'] = np.log(df['adj_close'] / df['prev_close'])
    df['ma_3'] = df['prev_close'].rolling(3).mean()
    df['ma_5'] = df['prev_close'].rolling(5).mean()
    df['ma_10'] = df['prev_close'].rolling(10).mean()
    df['volatility'] = df['return'].rolling(5).std()
    df['rsi'] = calculate_rsi(df['prev_close'])
    df['momentum_3'] = df['adj_close'] / df['adj_close'].shift(3) - 1
    df['momentum_5'] = df['adj_close'] / df['adj_close'].shift(5) - 1
    
    # c) Create Sentiment Momentum Features
    # Captures whether sentiment is improving or deteriorating
    df['finbert_momentum_3'] = df['finbert_compound'] - df['finbert_compound'].shift(3)
    df['vader_momentum_3'] = df['vader_compound'] - df['vader_compound'].shift(3)
    df['textblob_momentum_3'] = df['textblob_polarity'] - df['textblob_polarity'].shift(3)
    
    # d) Create Target Variable
    df['target_return'] = df['return'].shift(-1)
    
    return df

# 2. Apply to all tickers
print("Aggregating daily data to weekly and creating technical features...")
all_features_df = merged_daily_df.groupby('ticker').progress_apply(create_weekly_features)

# 3. Clean and Save
final_model_data = all_features_df.dropna()
final_model_data = final_model_data.reset_index() 
print(f"Created final tabular dataset with {len(final_model_data)} model-ready rows.")

# 4. Save to CSV
output_dir = base_dir / "1_IR_Data" / "5_Merged_Data"
output_dir.mkdir(exist_ok=True)
csv_path = output_dir / "model_ready_features.csv"
final_model_data.to_csv(csv_path, index=False)
print(f"\nSaved tabular (weekly) data to: {csv_path}")

# 5. Save to Database
print("Saving tabular data to 'model_features' table...")
try:
    # Rollback any pending transactions first
    try:
        engine.dispose()
    except:
        pass
    
    # Recreate engine connection
    engine = create_engine(db_url)
    
    # Convert date to string before saving
    final_model_data_db = final_model_data.copy()
    final_model_data_db['date'] = pd.to_datetime(final_model_data_db['date']).dt.strftime('%Y-%m-%d')
    
    final_model_data_db.to_sql(
        'model_features',
        con=engine,
        if_exists='replace',
        index=False,
        chunksize=1000
    )
    print("Successfully wrote features to database!")
except Exception as e:
    print(f"Error writing to database: {e}")
    print("Data was still saved to CSV successfully.")

# 6. Display Feature Summary
print("\n=== Feature Summary ===")
print(f"Total features: {len(final_model_data.columns)}")
print("\nFeature columns:")
feature_cols = [col for col in final_model_data.columns if col not in ['ticker', 'date', 'target_return']]
print(f"  Price/Technical: {[c for c in feature_cols if any(x in c for x in ['close', 'volume', 'return', 'ma_', 'rsi', 'momentum', 'volatility'])]}")
print(f"  Sentiment Base: {[c for c in feature_cols if any(x in c for x in ['textblob_polarity', 'vader_compound', 'finbert_compound']) and 'volatility' not in c and 'momentum' not in c]}")
print(f"  Sentiment Advanced: {[c for c in feature_cols if any(x in c for x in ['momentum', 'volatility', 'freshness', 'news_count', 'days_since'])]}")
print(f"\nTarget variable: target_return")

Creating Weekly Tabular Dataset.
Aggregating daily data to weekly and creating technical features...


  0%|          | 0/4041 [00:00<?, ?it/s]

Created final tabular dataset with 1855010 model-ready rows.

Saved tabular (weekly) data to: c:\Users\18kyu\Desktop\Unishit\IR\1_IR_Data\5_Merged_Data\model_ready_features.csv
Saving tabular data to 'model_features' table...
Successfully wrote features to database!

=== Feature Summary ===
Total features: 28

Feature columns:
  Price/Technical: ['adj_close', 'volume', 'textblob_polarity_volatility', 'vader_compound_volatility', 'finbert_compound_volatility', 'prev_close', 'return', 'log_return', 'ma_3', 'ma_5', 'ma_10', 'volatility', 'rsi', 'momentum_3', 'momentum_5', 'finbert_momentum_3', 'vader_momentum_3', 'textblob_momentum_3']
  Sentiment Base: ['textblob_polarity', 'vader_compound', 'finbert_compound']
  Sentiment Advanced: ['news_count', 'days_since_news', 'sentiment_freshness', 'textblob_polarity_volatility', 'vader_compound_volatility', 'finbert_compound_volatility', 'volatility', 'momentum_3', 'momentum_5', 'finbert_momentum_3', 'vader_momentum_3', 'textblob_momentum_3']

Ta

In [5]:
# Cell 5: Create Daily Sequence Datasets
print("\n Creating Daily Sequence Datasets.")

# 1. Define Daily Feature Function
def create_daily_target(group):
    """Creates the target variable for sequence models"""
    df = group.copy()
    df['return'] = df['close'].pct_change()
    df['target_return'] = df['return'].shift(-1)
    return df

print("Creating daily target variable.")
daily_features_df = merged_daily_df.groupby('ticker').progress_apply(create_daily_target)
daily_features_df = daily_features_df.dropna()
daily_features_df = daily_features_df.reset_index(drop=True)

# 2. Save one CSV file per ticker
output_seq_dir = base_dir / "1_IR_Data" / "6_Sequence_Data"
output_seq_dir.mkdir(exist_ok=True)
print(f"Saving individual ticker files to {output_seq_dir}.")

tickers_with_data = daily_features_df['ticker'].unique()

for ticker in tqdm(tickers_with_data, desc="Saving sequence files"):
    ticker_df = daily_features_df[daily_features_df['ticker'] == ticker]
    
    if len(ticker_df) > 50:
        file_path = output_seq_dir / f"{ticker}.csv"
        ticker_df.to_csv(file_path, index=False)

print(f"Saved {len(tickers_with_data)} individual ticker files for sequence modeling.")


 Creating Daily Sequence Datasets.
Creating daily target variable.


  0%|          | 0/4041 [00:00<?, ?it/s]

Saving individual ticker files to c:\Users\18kyu\Desktop\Unishit\IR\1_IR_Data\6_Sequence_Data.


Saving sequence files:   0%|          | 0/4041 [00:00<?, ?it/s]

Saved 4041 individual ticker files for sequence modeling.


In [6]:
# Cell 6: Save Daily Merged Data to Database
print("\n Saving Daily Merged Data to Database.")

try:
    merged_daily_df['date'] = merged_daily_df['date'].dt.strftime('%Y-%m-%d')        
    print("Writing 'data_daily_merged' to the database.")

    chunk_size = 5000 
    total_chunks = math.ceil(len(merged_daily_df) / chunk_size)
    chunks = np.array_split(merged_daily_df, total_chunks)
    
    # Write the first chunk (replace)
    first_chunk = chunks[0]
    first_chunk.to_sql(
        'data_daily_merged', 
        con=engine, 
        if_exists='replace', 
        index=False, 
        chunksize=1000
    )
    
    # Write remaining chunks (append)
    for chunk in tqdm(chunks[1:], desc="Writing Daily Data", total=total_chunks, initial=1):
        chunk.to_sql(
            'data_daily_merged', 
            con=engine, 
            if_exists='append', 
            index=False, 
            chunksize=1000
        )
    
    print("Successfully wrote 'data_daily_merged' to the database!")
    
except Exception as e:
    print(f"Error writing daily data to database: {e}")


 Saving Daily Merged Data to Database.
Writing 'data_daily_merged' to the database.


Writing Daily Data:   0%|          | 1/1798 [00:00<?, ?it/s]

Successfully wrote 'data_daily_merged' to the database!
